In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [2]:
#!wget https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats-model/dogs_cats_10_0.687.h5 -O dogs_cats_model.h5

In [3]:
model = keras.models.load_model('dogs_cats_model.h5')

In [4]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_input(x):
    #x /= 127.5
    #x -= 1.
    x /= 255
    return x

In [5]:
url = 'https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg'
image_size = (150, 150)

img = download_image(url)
img = prepare_image(img, target_size=image_size)

In [6]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)
X

array([[[[0.7058824 , 0.77254903, 0.7490196 ],
         [0.69411767, 0.7607843 , 0.7294118 ],
         [0.6313726 , 0.69803923, 0.6666667 ],
         ...,
         [0.7647059 , 0.8509804 , 0.8392157 ],
         [0.68235296, 0.74509805, 0.74509805],
         [0.6745098 , 0.76862746, 0.7764706 ]],

        [[0.5254902 , 0.5921569 , 0.5686275 ],
         [0.7372549 , 0.8039216 , 0.78039217],
         [0.60784316, 0.6745098 , 0.6431373 ],
         ...,
         [0.7921569 , 0.8666667 , 0.85882354],
         [0.6313726 , 0.7019608 , 0.6392157 ],
         [0.7882353 , 0.8784314 , 0.85490197]],

        [[0.6862745 , 0.7490196 , 0.7372549 ],
         [0.61960787, 0.6862745 , 0.6627451 ],
         [0.64705884, 0.7137255 , 0.68235296],
         ...,
         [0.6       , 0.65882355, 0.64705884],
         [0.627451  , 0.69411767, 0.654902  ],
         [0.7058824 , 0.79607844, 0.77254903]],

        ...,

        [[0.5803922 , 0.61960787, 0.58431375],
         [0.49019608, 0.5254902 , 0.5058824 ]

In [7]:
X.shape

(1, 150, 150, 3)

In [8]:
X[0][0][0][0]

0.7058824

In [9]:
pred = model.predict(X)

In [10]:
classes = [
    'dogs',
    'cats'
]

dict(zip(classes, pred[0]))

{'dogs': 0.77049136}

In [11]:
## Convert Keras to TF-Lite

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('dogs_cats_model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/wk/yt_rdhdx7mqbrqjqngjkkqx00000gn/T/tmpau0u9a9g/assets


In [13]:
#!ls -lh

In [14]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/


In [15]:
!pip install tflite_runtime-2.5.0-cp38-cp38-macosx_10_13_x86_64.whl

Processing ./tflite_runtime-2.5.0-cp38-cp38-macosx_10_13_x86_64.whl
tflite-runtime is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [16]:
#import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite

In [17]:
interpreter = tflite.Interpreter(model_path='dogs_cats_model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [18]:
output_index

13

In [19]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
pred = interpreter.get_tensor(output_index)

In [20]:
classes = [
    'dogs',
    'cats'
]

dict(zip(classes, pred[0]))

{'dogs': 0.770491}